<a href="https://colab.research.google.com/github/benjamin-carter/time_series_walmart/blob/master/walmart_ts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 7: Forecasting Weekly Sales for Walmart Store

Connect to Google Drive

In [0]:
from google.colab import drive

drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [0]:
ls "/content/gdrive/My Drive/Current"

Add necessary packages for data manipulation, dataframe handling, plotting,  time series modelling, and LSTM creation and training

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
  
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.arima_model import ARIMA
import datetime

import seaborn as sns
from IPython.display import display, HTML, display_html

import tensorflow as tf
import os

plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams['axes.grid'] = False

Upload Data from Google Drive. Upload and edit testing dataset (empty)

In [0]:
df = pd.read_csv("/content/gdrive/My Drive/Current/walmart_train.csv")
df_test = pd.read_csv("/content/gdrive/My Drive/Current/walmart_test.csv")

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

df_example = df_by_store(df, 1)
df_test_ = df_by_store(df_test, 1)
df_test_ = df_test_['2012-11-02':'2013-02-01']

# Decomposition
Remove end-of-year pulse, overall trend, and saw-tooth function found in the original data

In [0]:
mean10 = df_example['ts']['2010-02-11':'2010-11-18'].to_numpy().mean()
mean11 = df_example['ts']['2011-02-05':'2011-11-12'].to_numpy().mean()
temp10 = df_example['ts']['2010-11-25':'2011-02-03'].to_numpy()
temp11 = df_example['ts']['2011-11-19':'2012-01-27'].to_numpy()

temp10_p = temp10/mean10
temp11_p = temp11/mean11

temp_tot_p = .5*(temp10_p+temp11_p)

df_example['ts']['2010-11-25':'2011-02-03'] = df_example['ts']['2010-11-25':'2011-02-03'].to_numpy() - mean10 * (temp_tot_p-1)
df_example['ts']['2011-11-19':'2012-01-27'] = df_example['ts']['2011-11-19':'2012-01-27'].to_numpy() - mean11 * (temp_tot_p-1)

Y = df_example['ts'].to_numpy().reshape(len(df_example['ts']),1)
X = np.arange(len(Y)).reshape((len(Y),1))
regressor = LinearRegression()  
regressor.fit(X,Y)
growth = regressor.coef_
init_int = regressor.intercept_
df_example['ts_trend'] = regressor.intercept_ + X * regressor.coef_
df_example['ts_trend_adj'] = df_example['ts'] - df_example['ts_trend']

Y = df_example['ts_trend_adj'].to_numpy().reshape(len(df_example['ts']),1)
X = np.arange(len(Y)).reshape((len(Y),1))
regressor = LinearRegression()  
regressor.fit(X,Y)
overall_trend = regressor.intercept_ + X * regressor.coef_

len10 = len(df_example['ts_trend_adj']['2010-02-05':'2010-11-19'])
Y = df_example['ts_trend_adj']['2010-02-05':'2010-11-19'].to_numpy().reshape(len10, 1)

# len10 = len(df_example['ts_trend_adj']['2010-02-05':'2011-01-29'])
# Y = df_example['ts_trend_adj']['2010-02-05':'2011-01-29'].to_numpy().reshape(len10, 1)
X_10 = np.arange(len(Y)).reshape((len(Y),1))
regressor = LinearRegression()  
regressor.fit(X_10,Y)
rate_2010 = regressor.coef_
int_2010 = regressor.intercept_

len11 = len(df_example['ts_trend_adj']['2010-11-26':'2011-11-18'])
Y = df_example['ts_trend_adj']['2010-11-26':'2011-11-18'].to_numpy().reshape(len11, 1)

# len11 = len(df_example['ts_trend_adj']['2011-02-04':'2012-01-28'])
# Y = df_example['ts_trend_adj']['2011-02-04':'2012-01-28'].to_numpy().reshape(len11, 1)
X_11 = np.arange(len(Y)).reshape((len(Y),1))
regressor = LinearRegression()  
regressor.fit(X_11,Y)
rate_2011 = regressor.coef_
int_2011 = regressor.intercept_

len12 = len(df_example['ts_trend_adj']['2011-11-25':])
Y = df_example['ts_trend_adj']['2011-11-25':].to_numpy().reshape(len12,1)

# len12 = len(df_example['ts_trend_adj']['2012-02-03':])
# Y = df_example['ts_trend_adj']['2012-02-03':].to_numpy().reshape(len12,1)
X_12 = np.arange(len(Y)).reshape((len(Y),1))
regressor = LinearRegression()  
regressor.fit(X_12,Y)
rate_2012 = regressor.coef_
int_2012 = regressor.intercept_

overall_rate = (1/3) * (rate_2010 + rate_2011+ rate_2012)
overall_int = (1/3) * (int_2010 + int_2011 + int_2012)
trend_2010 = overall_int + X_10 * overall_rate
trend_2011 = overall_int + X_11 * overall_rate
trend_2012 = overall_int + X_12 * overall_rate

saw_tooth = np.zeros((len(df_example),1))
saw_tooth[:len10] = overall_trend[:len10] + trend_2010
saw_tooth[len10:(len10 + len11)] = overall_trend[len10:(len10 + len11)] + trend_2011
saw_tooth[(len10 + len11):] = overall_trend[(len10 + len11):] + trend_2012
df_example['saw_tooth'] = saw_tooth
df_example['ts_saw_adj'] = df_example['ts_trend_adj'] - df_example['saw_tooth']

Y = df_example['ts_saw_adj'].to_numpy().reshape(len(df_example['ts_saw_adj']),1)
X = np.arange(len(Y)).reshape((len(Y),1))
regressor = LinearRegression()  
regressor.fit(X,Y)
growth2 = regressor.coef_
init_int2 = regressor.intercept_
df_example['ts_st_trend'] = regressor.intercept_ + X * regressor.coef_
df_example['ts_trend_adj_v2'] = df_example['ts_saw_adj'] - df_example['ts_st_trend']

## Create plot for each decomposed function to illustrate process and result

In [0]:
df_example['ts_moving_avg'] = df_example['ts'].rolling(window = 7, center= False).mean()
plt.figure(figsize=(10,5))
plt.plot(df_example['ts'], linewidth = 3)
plt.title("Store #1: Weekly Sales", fontsize = 20)
plt.xlabel("Date", fontsize = 20)
plt.ylabel("$M", fontsize = 20)
plt.show()
plt.clf()


plt.figure(figsize=(10,5))
plt.plot(df_example['ts']['2010-11-18':'2011-02-10'], linewidth = 3)
plt.plot(df_example['ts']['2011-11-12':'2012-02-04'], linewidth = 3)
plt.title("Nov-Jan Pulse", fontsize = 20)
plt.xlabel("Date", fontsize = 20)
plt.ylabel("$M", fontsize = 20)
plt.show()
plt.clf()


plt.figure(figsize=(10,5))
plt.title("Upward Trend", fontsize = 20)
plt.xlabel("Date", fontsize = 20)
plt.ylabel("$M", fontsize = 20)
plt.plot(df_example['ts']['2010-02-05':'2011-01-28'])
plt.plot(df_example['ts']['2011-02-04':'2012-01-27'])
plt.plot(df_example['ts']['2012-02-03':'2012-10-26'])
plt.plot(df_example['ts_trend'])
plt.show()
plt.clf()

plt.figure(figsize=(10,5))
plt.title("Annual Decrease", fontsize = 20)
plt.xlabel("Date", fontsize = 20)
# plt.ylabel("$M", fontsize = 20)
plt.plot(df_example['ts_trend_adj'])
plt.plot(df_example['saw_tooth'])
plt.show()
plt.clf()

plt.figure(figsize=(10,5))
plt.title("Final Noise Signal", fontsize = 20)
plt.xlabel("Date", fontsize = 20)
# plt.ylabel("$M", fontsize = 20)
plt.plot(df_example['ts_trend_adj_v2'])
plt.show()
plt.clf()

## Time Series Analysis

Check stationarity for final signal

In [0]:
test_stationarity(df_example, 'ts_trend_adj_v2')

Run decomposition process (unnecessary)

In [0]:
decomposition = seasonal_decompose(df_example['ts'], freq = 52)

df_example.loc[:,'trend'] = decomposition.trend
df_example.loc[:,'seasonal'] = decomposition.seasonal
df_example.loc[:,'residual'] = decomposition.resid

plot_decomposition(df = df_example, ts= 'ts',trend='trend', seasonal='seasonal',residual='residual')

test_stationarity(df_example.dropna(), ts='residual')

Run ACF and PACF analysis. Plot results with bar graph.

In [0]:
lag_acf=acf(np.array(df_example['ts_trend_adj_v2']),nlags=10)
lag_pacf=pacf(np.array(df_example['ts_trend_adj_v2']),nlags=10)

lag_acf = pd.DataFrame(lag_acf)
lag_acf.plot(kind = 'bar')
plt.title("ACF", fontsize = 20)
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(143),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(143),linestyle='--',color='gray')
lag_pacf = pd.DataFrame(lag_pacf)
lag_pacf.plot(kind = 'bar')
plt.title("PACF", fontsize = 20)
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(143),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(143),linestyle='--',color='gray')

Run ARIMA model with (p,d,q) found in analysis. Create forecast for 14 weeks ahead.

In [0]:
model=ARIMA(df_example['ts_trend_adj_v2'], order=(4,0,5))
results_arima=model.fit(disp=-1)
decomp_last12 = results_arima.fittedvalues['2012-08-10':]
arima_pred = results_arima.predict(start = '2012-11-02', end = '2013-02-01')
arima_pred = arima_pred.to_numpy()
conf_int = results_arima.forecast(14,alpha=.05)[2]
# full_pred = np.column_stack((arima_pred, conf_int))
# plt.plot(full_pred[:,0], label = "Prediction")
# plt.plot(full_pred[:,1], label = "95% Lower bound")
# plt.plot(full_pred[:,2], label = "95% Upper bound")
# plt.legend(loc='lower left')


### Backwards Engineer results from signal analysis to be comparable with original data

In [0]:
arima_pred = arima_pred.reshape(len(arima_pred),)
df_example['ts_last12_decomp'] = df_example['ts']
df_example['ts_last12_decomp']['2012-08-10':'2012-10-26'] = decomp_last12.to_numpy()
Y = df_example['ts_saw_adj'].to_numpy().reshape(len(df_example['ts_saw_adj']),1)
X = np.arange(len(Y)).reshape((len(Y),1))
regressor = LinearRegression()  
regressor.fit(X,Y)
X_test = np.arange(len(Y),len(Y)+ 14).reshape(14,)
temp =  regressor.intercept_ + X_test * regressor.coef_ + arima_pred
temp = temp.reshape(14,)
df_test_['ts_bef_saw'] = temp
temp =  regressor.intercept_ + X_test * regressor.coef_ + conf_int[:,0]
temp = temp.reshape(14,)
df_test_['ts_bef_saw_down'] = temp
temp =  regressor.intercept_ + X_test * regressor.coef_ + conf_int[:,1]
temp = temp.reshape(14,)
df_test_['ts_bef_saw_up'] = temp
temp =  regressor.intercept_ + X[-12:] * regressor.coef_
temp = temp.reshape(12,)
df_example['ts_last12_decomp']['2012-08-10':'2012-10-26'] = df_example['ts_last12_decomp']['2012-08-10':'2012-10-26'] + temp

Y = df_example['ts_trend_adj'].to_numpy().reshape(len(df_example['ts']),1)
X = np.arange(len(Y)).reshape((len(Y),1))
regressor = LinearRegression()  
regressor.fit(X,Y)
X_test = np.arange(len12,len12+ 14).reshape(14,1)
future_14 = overall_int + X_test * overall_rate
future_12 = overall_int + X[-12:] * overall_rate
overall_future = regressor.intercept_ + X_test * regressor.coef_
overall_future12 = regressor.intercept_ + X[-12:] * regressor.coef_
df_test_['ts_bef_OT'] = df_test_['ts_bef_saw'] + future_14.reshape(14,)
df_test_['ts_bef_trend'] = df_test_['ts_bef_OT'] + overall_future.reshape(14,)
df_test_['ts_bef_OT_down'] = df_test_['ts_bef_saw_down'] + future_14.reshape(14,)
df_test_['ts_bef_OT_up'] = df_test_['ts_bef_saw_up'] + future_14.reshape(14,)
df_test_['ts_bef_trend_down'] = df_test_['ts_bef_OT_down'] + overall_future.reshape(14,)
df_test_['ts_bef_trend_up'] = df_test_['ts_bef_OT_up'] + overall_future.reshape(14,)
df_example['ts_last12_decomp']['2012-08-10':'2012-10-26'] = df_example['ts_last12_decomp']['2012-08-10':'2012-10-26'] + future_12.reshape(12,)
df_example['ts_last12_decomp']['2012-08-10':'2012-10-26'] = df_example['ts_last12_decomp']['2012-08-10':'2012-10-26'] + overall_future12.reshape(12,)

Y = df_example['ts'].to_numpy().reshape(len(df_example['ts']),1)
X = np.arange(len(Y)).reshape((len(Y),1))
X_test = np.arange(len(Y),len(Y)+ 14).reshape(14,)
regressor = LinearRegression()  
regressor.fit(X,Y)
temp = regressor.intercept_ + X_test * regressor.coef_
temp = temp.reshape(14,)
df_test_['ts_bef_pulse'] = df_test_['ts_bef_trend'] + temp
df_test_['ts_bef_pulse_down'] = df_test_['ts_bef_trend_down'] + temp
df_test_['ts_bef_pulse_up'] = df_test_['ts_bef_trend_up'] + temp
temp = regressor.intercept_ + X[-12:] * regressor.coef_
temp = temp.reshape(12,)
df_example['ts_last12_decomp']['2012-08-10':'2012-10-26'] = df_example['ts_last12_decomp']['2012-08-10':'2012-10-26'] + temp

mean12 = df_example['ts']['2012-02-04':'2012-10-26'].to_numpy().mean()
temp_tot_p = .5*(temp10_p+temp11_p)
df_test_['ts_predict'] = df_test_['ts_bef_pulse']
df_test_['ts_predict']['2012-11-23':'2013-01-25'] = df_test_['ts_predict']['2012-11-23':'2013-01-25'] + mean12 * (temp_tot_p-1)
df_test_['ts_predict_down'] = df_test_['ts_bef_pulse_down']
df_test_['ts_predict_down']['2012-11-23':'2013-01-25'] = df_test_['ts_predict_down']['2012-11-23':'2013-01-25'] + mean12 * (temp_tot_p-1)
df_test_['ts_predict_up'] = df_test_['ts_bef_pulse_up']
df_test_['ts_predict_up']['2012-11-23':'2013-01-25'] = df_test_['ts_predict_up']['2012-11-23':'2013-01-25'] + mean12 * (temp_tot_p-1)

# df_test_['ts_predict_down'] = df_test_['ts_predict_down'] - df_example['ts'].std()
# df_test_['ts_predict_up'] = df_test_['ts_predict_up'] + df_example['ts'].std()



### Neural Network modelling

Download a fresh batch of the same data, in order to compare the results more easily

In [0]:
df = pd.read_csv("/content/gdrive/My Drive/Current/walmart_train.csv")
df_test = pd.read_csv("/content/gdrive/My Drive/Current/walmart_test.csv")

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

df_example2 = df_by_store(df, 1)
df_test_2 = df_by_store(df_test, 1)
df_test_2 = df_test_['2012-11-02':'2013-02-01']

Create LSTM data and model specifically for the 14 week blind forecast and analysis

In [0]:
ts_np = df_example2['ts'].to_numpy()
ts_mean = ts_np.mean()
ts_std = ts_np.std()
ts_np = (ts_np - ts_mean) / ts_std
zero_14 = np.zeros((1,15))
ts_np = np.append(ts_np,zero_14)
ts_ord, ts_test = univariate_data(ts_np, start_index=0, end_index=None, history_size = 50, target_size = 14)


BATCH_SIZE = 64
BUFFER_SIZE = 10000

train_univariate = tf.data.Dataset.from_tensor_slices((ts_ord[:-15], ts_test[:-15]))
train_univariate = train_univariate.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

val_univariate = tf.data.Dataset.from_tensor_slices((ts_ord[-15:-14], ts_test[-15:-14]))
val_univariate = val_univariate.batch(BATCH_SIZE).repeat()

future_univ = tf.data.Dataset.from_tensor_slices((ts_ord[-1:], ts_test[-1:]))
future_univ = future_univ.batch(BATCH_SIZE).repeat()

simple_lstm_model_f = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(32, return_sequences = True, input_shape=ts_ord.shape[-2:]),
    tf.keras.layers.LSTM(16, activation = 'relu'),
    tf.keras.layers.Dense(14)
])

simple_lstm_model_f.compile(optimizer='adam', loss='mae')

EVALUATION_INTERVAL = 100
EPOCHS = 10

simple_lstm_model_f.fit(train_univariate, epochs=EPOCHS,
                      steps_per_epoch=EVALUATION_INTERVAL,
                      validation_data=val_univariate, validation_steps=12)

Run results and calculate metrics for the 14 week blind forecast

In [0]:
for x, y in future_univ.take(1):
  lstm_fc14 = simple_lstm_model_f.predict(x)[0]
lstm_fc14 = lstm_fc14.reshape(14,1)
lstm_fc14 = lstm_fc14 *ts_std + ts_mean
x, y = next(iter(future_univ))

for x, y in future_univ.take(1):
  plot = show_plot([x[0].numpy(),simple_lstm_model_f.predict(x)[0]], 0, 'Simple LSTM model')
  plot.show()

df_test_2['LSTM_fc'] =lstm_fc14

Create LSTM data and model specifcally for the 12 week known forecast and analysis

In [0]:
ts_np = df_example2['ts'].to_numpy()
ts_mean = ts_np.mean()
ts_std = ts_np.std()
ts_np = (ts_np - ts_mean) / ts_std
ts_ord, ts_test = univariate_data(ts_np, start_index=0, end_index=None, history_size = 52, target_size = 12)

BATCH_SIZE = 64
BUFFER_SIZE = 10000

train_univariate = tf.data.Dataset.from_tensor_slices((ts_ord[:-1], ts_test[:-1]))
train_univariate = train_univariate.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

val_univariate_12 = tf.data.Dataset.from_tensor_slices((ts_ord[-1:], ts_test[-1:]))
val_univariate_12 = val_univariate_12.batch(BATCH_SIZE).repeat()

simple_lstm_model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(32, return_sequences = True, input_shape=ts_ord.shape[-2:]),
    tf.keras.layers.LSTM(16, activation = 'relu'),
    tf.keras.layers.Dense(12)
])

simple_lstm_model.compile(optimizer='adam', loss='mae')

EVALUATION_INTERVAL = 100
EPOCHS = 10

simple_lstm_model.fit(train_univariate, epochs=EPOCHS,
                      steps_per_epoch=EVALUATION_INTERVAL,
                      validation_data=val_univariate_12, validation_steps=12)

Run results and calculate metrics for the 12 week known forecast

In [0]:
for x, y in val_univariate_12.take(1):
  lstm_fc = simple_lstm_model.predict(x)[0]
lstm_fc = lstm_fc.reshape(12,1)
lstm_fc = lstm_fc *ts_std + ts_mean
x, y = next(iter(val_univariate_12))
y = np.array(y).reshape(12,1)
y = y *ts_std + ts_mean
lstm_fc = lstm_fc.reshape(12,)
df_example2['ts_last12_lstm'] = df_example['ts']
df_example2['ts_last12_lstm']['2012-08-10':'2012-10-26'] = lstm_fc.reshape(12,)
lstm_fc = lstm_fc.reshape(12,1)
rmse = np.sqrt(sum((lstm_fc-y)**2))
rmse = np.asscalar(rmse)
rmse = round(rmse)

for x, y in val_univariate_12.take(1):
  plot = show_plot([x[0].numpy(),  simple_lstm_model.predict(x)[0], y[0].numpy()],
                    0, 'Simple LSTM model')
  plot.show()



## Attempt to train single model with different groups of stores

In [0]:
# Create model before
simple_lstm_model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(64, return_sequences = True, input_shape=(104,26)),
    tf.keras.layers.LSTM(32, activation = 'relu'),
    tf.keras.layers.Dense(26)
])
simple_lstm_model.compile(optimizer='adam', loss='mae')

all_group = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45]

size_group = [14,41,39,34,40,8,26,21,25,10,9,15,18,22,45,23,12,35,29,1]
typeA_group = [13,11,28,4,27,19,24,31,20,32,6,2,14,41,39,34,40,8,26,1]
dept_group = [2,4,6,13,34,40,1]
all_groups = []
all_groups.append(all_group)
all_groups.append(size_group)
all_groups.append(typeA_group)
all_groups.append(dept_group)
for j in range(len(all_groups)):
  for i in range(len(all_groups[j])):
    df_store = df_by_store(df, all_groups[j][i])
    ts_np = df_example['ts'].to_numpy()
    days = len(ts_np)
    # Standardize data
    ts_np_norm = (ts_np - ts_np.mean()) / ts_np.std()
    # Break up windows fro training and testing
    ts_ord, ts_test = univariate_data(ts_np_norm, start_index=0, end_index=None, history_size = 104 , target_size = 26)
    #Create tensors for training and validation
    BATCH_SIZE = 64
    BUFFER_SIZE = 10000
    train_univariate = tf.data.Dataset.from_tensor_slices((ts_ord[:-1], ts_test[:-1]))
    train_univariate = train_univariate.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
    val_univariate = tf.data.Dataset.from_tensor_slices((ts_ord[-1:], ts_test[-1:]))
    val_univariate = val_univariate.batch(BATCH_SIZE).repeat()
    # Train model
    EVALUATION_INTERVAL = 100 
    EPOCHS = 10
    simple_lstm_model.fit(train_univariate, epochs=EPOCHS,
                      steps_per_epoch=EVALUATION_INTERVAL,
                      validation_data=val_univariate, validation_steps=52)

df_store = df_by_store(df, 1)
ts_np = df_example['ts'].to_numpy()
days = len(ts_np)
# Standardize data
ts_np_norm = (ts_np - ts_np.mean()) / ts_np.std()
# Break up windows fro training and testing
ts_ord, ts_test = univariate_data(ts_np_norm, start_index=0, end_index=None, history_size = 104, target_size = 26)
#Create tensors for training and validation
BATCH_SIZE = 64
BUFFER_SIZE = 10000
train_univariate = tf.data.Dataset.from_tensor_slices((ts_ord[:-1], ts_test[:-1]))
train_univariate = train_univariate.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
val_univariate = tf.data.Dataset.from_tensor_slices((ts_ord[-1:], ts_test[-1:]))
val_univariate = val_univariate.batch(BATCH_SIZE).repeat()
# Train model
EVALUATION_INTERVAL = 100 
EPOCHS = 10
simple_lstm_model.fit(train_univariate, epochs=EPOCHS,
                      steps_per_epoch=EVALUATION_INTERVAL,
                      validation_data=val_univariate, validation_steps=12)
# Predict sales 
for x, y in val_univariate.take(1):
  sales_pred = simple_lstm_model.predict(x)[0]
# Convert back into true $ units
sales_pred = sales_pred * ts_np.std() + ts_np.mean()

df_example['ts_last12_all'] = df_example['ts']
df_example['ts_last12_all']['2012-08-10':'2012-10-26']  = sales_pred[-12:]

Create datasets and running them through the model for 14 blind forecast for the grouped store model

In [0]:
ts_np = df_example2['ts'].to_numpy()
ts_mean = ts_np.mean()
ts_std = ts_np.std()
ts_np = (ts_np - ts_mean) / ts_std
zero_27 = np.zeros((1,27))
ts_np = np.append(ts_np,zero_27)
ts_ord, ts_test = univariate_data(ts_np, start_index=0, end_index=None, history_size = 104, target_size = 26)


BATCH_SIZE = 64
BUFFER_SIZE = 10000

future_univ_all = tf.data.Dataset.from_tensor_slices((ts_ord[-1:], ts_test[-1:]))
future_univ_all = future_univ_all.batch(BATCH_SIZE).repeat()

for x, y in future_univ_all.take(1):
  temp = simple_lstm_model_all.predict(x)[0]
temp = temp * ts_std + ts_mean
df_test_['ts_pred_all'] = temp[:14]

# Prophet

Adding necessary packages

In [0]:
!pip install fbprophet
!pip install plotly
import numpy as np
import pandas as pd
from fbprophet import Prophet
from fbprophet.plot import plot_plotly
from fbprophet.plot import add_changepoints_to_plot

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.arima_model import ARIMA

import matplotlib.pyplot as plt
import seaborn as seabornInstance
import plotly.offline as py
 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
py.init_notebook_mode()
%matplotlib inline

Creating dataset that was compatible with Prophet functions

In [0]:
df_prophet = df_by_store_prophet(df, 1)
# df_prophet['ds'] = df_prophet.index
df_prophet['ds'] = df_prophet.index
df_prophet['ds'] = pd.to_datetime(df_prophet['ds'])
df_prophet.columns

Run Prophet modelling and get results for both y-fitted values and y-hat future values

In [0]:
prophet = Prophet()
prophet.fit(df_prophet)
future = prophet.make_future_dataframe(periods=14, freq='W')
forecast = prophet.predict(future)
fig = prophet.plot(forecast)
add_changepoints_to_plot(fig.gca(), prophet, forecast)

Store values to use in comparison with other models

In [0]:
temp_pro_14 = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']][-14:]
temp_pro_14np = temp_pro_14['yhat'].to_numpy()
df_test_['ts_proph'] = temp_pro_14np

temp_pro_12 = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']][-26:-14]
temp_pro_12np = temp_pro_12['yhat'].to_numpy()
df_example['ts_prophet'] = df_example['ts']
df_example['ts_prophet']['2012-08-10':'2012-10-26'] = temp_pro_12np


## Plot different comparisons of different model's results and metrics

In [0]:
plt.figure()
plt.plot(df_test_['ts_predict'], label = "Decomposition")
# plt.plot(df_test_['ts_predict_up'], label = "95% Upper bound")
# plt.plot(df_test_['ts_predict_down'], label = "95% Lower bound")
plt.plot(df_test_2['LSTM_fc'], label = "LSTM - Store 1")
# plt.plot(df_test_['ts_pred_all'], label = "LSTM - All" )
plt.plot(df_test_['ts_proph'], label = "Prophet")
plt.title("14 Week Blind Forecast")
plt.legend(loc='top right')
plt.show()
plt.clf()

decomp = df_example['ts_last12_decomp']['2012-08-10':'2012-10-26'].to_numpy()
lstm_rm = df_example2['ts_last12_lstm']['2012-08-10':'2012-10-26'].to_numpy()
actual_rm = df_example['ts']['2012-08-10':'2012-10-26'].to_numpy()

decomp_rmse = np.sqrt(sum((decomp-actual_rm)**2))
lstm_rmse = np.sqrt(sum((lstm_rm-actual_rm)**2))
decomp_rmse =  round(decomp_rmse)
lstm_rmse = round(lstm_rmse)
lstm_all = sales_pred[-12:]
lstmall_rmse = np.sqrt(sum((lstm_all-actual_rm)**2))
lstmall_rmse = round(lstmall_rmse)
temp_pro_np
proph_rmse = np.sqrt(sum((temp_pro_12np-actual_rm)**2))
proph_rmse = round(proph_rmse)

plt.figure()
plt.plot(df_example['ts']['2012-08-10':'2012-10-26'], label = "Actual")
# plt.plot(df_example['ts_last12_decomp']['2012-08-10':'2012-10-26'] , label = "Decomp. RMSE: %s" %decomp_rmse)
plt.plot(df_example2['ts_last12_lstm']['2012-08-10':'2012-10-26'], label = "LSTM - Store1. RMSE: %s" %lstm_rmse)
# plt.plot(df_example['ts_last12_all']['2012-08-10':'2012-10-26'], label = "LSTM - All. RMSE: %s" %lstmall_rmse)
plt.plot(df_example['ts_prophet']['2012-08-10':'2012-10-26'], label = "Prophet. RMSE: %s" %proph_rmse)
plt.title("12 Week Known Forecast")
plt.legend(loc='lower left')
plt.show()
plt.clf()


## Global Functions

In [0]:
def df_by_store(df, store_no):
  df_store = df[df['Store'] == store_no]
  df_store = df_store.rename(columns = {'Date': 'ds', 'Weekly_Sales':'ts'})
  df_example = df_store.groupby(by = 'ds').agg({'ts':'sum'})
  df_example.index = pd.to_datetime(df_example.index) 
  df_example = df_example.asfreq(freq='7D')
  df_example = df_example.sort_index(ascending = True)
  df_example = df_example.fillna(value=0)
  return df_example

def df_by_dept(df, dept_no):
  df_store = df[df['Dept'] == dept_no]
  df_store = df_store.rename(columns = {'Date': 'ds', 'Weekly_Sales':'ts'})
  df_example = df_store.groupby(by = 'ds').agg({'ts':'mean'})
  df_example.index = pd.to_datetime(df_example.index) 
  df_example = df_example.asfreq(freq='7D')
  df_example = df_example.sort_index(ascending = True)
  df_example = df_example.fillna(value=0)
  return df_example


def test_stationarity(df, ts):
    """
    Test stationarity using moving average statistics and Dickey-Fuller test
    Source: https://www.analyticsvidhya.com/blog/2016/02/time-series-forecasting-codes-python/
    """
    
    # Determing rolling statistics
    rolmean = df[ts].rolling(window = 12, center = False).mean()
    rolstd = df[ts].rolling(window = 12, center = False).std()
    
    # Plot rolling statistics:
    orig = plt.plot(df[ts], 
                    color = 'blue', 
                    label = 'Original')
    mean = plt.plot(rolmean, 
                    color = 'red', 
                    label = 'Rolling Mean')
    std = plt.plot(rolstd, 
                   color = 'black', 
                   label = 'Rolling Std')
    plt.legend(loc = 'best')
    plt.title('Rolling Mean & Standard Deviation for %s' %(ts), fontsize = 15)
    plt.xticks(rotation = 45)
    plt.show(block = False)
    plt.close()
    
    # Perform Dickey-Fuller test:
    # Null Hypothesis (H_0): time series is not stationary
    # Alternate Hypothesis (H_1): time series is stationary
    print ('Results of Dickey-Fuller Test:')
    dftest = adfuller(df[ts], 
                      autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], 
                         index = ['Test Statistic',
                                  'p-value',
                                  '# Lags Used',
                                  'Number of Observations Used'])
    for key, value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)

def plot_decomposition(df, ts,trend,seasonal, residual):

  f, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2, figsize = (15, 5), sharex = True)

  ax1.plot(df[ts], label = 'Original')
  ax1.legend(loc = 'best')
  ax1.tick_params(axis = 'x', rotation = 45)

  ax2.plot(df[trend], label = 'Trend')
  ax2.legend(loc = 'best')
  ax2.tick_params(axis = 'x', rotation = 45)

  ax3.plot(df[seasonal],label = 'Seasonality')
  ax3.legend(loc = 'best')
  ax3.tick_params(axis = 'x', rotation = 45)

  ax4.plot(df[residual], label = 'Residuals')
  ax4.legend(loc = 'best')
  ax4.tick_params(axis = 'x', rotation = 45)
  plt.tight_layout()

  plt.subtitle('Signal Decomposition of  %s' %(ts), x =0.5, y= 1.05, fontsize = 18)
  plt.show()

def plot_acf_pacf(df, ts):
  """
  Plot auto-correlation function (ACF) and partial auto-correlation (PACF) plots
  """
  f, (ax1, ax2) = plt.subplots(1,2, figsize = (10, 5)) 

  #Plot ACF: 

  ax1.plot(lag_acf)
  ax1.axhline(y=0,linestyle='--',color='gray')
  ax1.axhline(y=-1.96/np.sqrt(len(df[ts])),linestyle='--',color='gray')
  ax1.axhline(y=1.96/np.sqrt(len(df[ts])),linestyle='--',color='gray')
  ax1.set_title('Autocorrelation Function for %s' %(ts))

  #Plot PACF:
  ax2.plot(lag_pacf)
  ax2.axhline(y=0,linestyle='--',color='gray')
  ax2.axhline(y=-1.96/np.sqrt(len(df[ts])),linestyle='--',color='gray')
  ax2.axhline(y=1.96/np.sqrt(len(df[ts])),linestyle='--',color='gray')
  ax2.set_title('Partial Autocorrelation Function for %s' %(ts))
  
  plt.tight_layout()
  plt.show()
  plt.close()
  
  return

def run_arima(df,ts,p,d,q):
  
  model=ARIMA(df[ts], order=(p,d,q))
  results_arima=model.fit(disp=-1)

  len_results = len(results_arima.fittedvalues)
  ts_modified = df[ts][-len_results:]

  rss  = sum((results_arima.fittedvalues - ts_modified)**2)
  rmse = np.sqrt(rss/ len(df[ts]))

  plt.figure()
  plt.plot(df[ts], label = "Actual")
  plt.plot(results_arima.fittedvalues, color='red', label = "Fitted")
  plt.legend(loc = 'best')
  plt.title('RMSE: %.4f'% rmse, fontsize=20)

  plt.show()

  return results_arima

def days_between(d1, d2):
  d1 = datetime.strptime(d1, "%Y-%m-%d")
  d2 = datetime.strptime(d2, "%Y-%m-%d")
  return abs((d2-d1).days + 1)

def daily_forecast (df, holidays, growth, holidays_prior_scale = 10, n_changepoints = 25, changepoint_prior_scale = 0.05, 
                   changepoints = None, interval_width = 0.8, mcmc_samples = 1, future_num_points = 10, daily_seasonality = True):
  df_c = df.copy()
  m = Prophet(growth = growth,
              n_changepoints = n_changepoints,
              changepoint_prior_scale = changepoint_prior_scale,
              changepoints = changepoints,
              holidays = holidays,
              holidays_prior_scale = holidays_prior_scale,
              interval_width = interval_width,
              mcmc_samples = mcmc_samples, 
              daily_seasonality = daily_seasonality)
  m.fit(df_c)
  future = m.make_future_dataframe(periods = future_num_points)
  forecast_1 = m.predict(future)

  m.plot(forecast_1);
  m.plot_components(forecast_1)

  return forecast_1

def univariate_data(dataset, start_index, end_index, history_size, target_size):
  data = []
  labels = []

  start_index = start_index + history_size
  if end_index is None:
    end_index = len(dataset) - target_size

  for i in range(start_index, end_index):
    indices = range(i-history_size, i)
    # Reshape data from (history_size,) to (history_size, 1)
    data.append(np.reshape(dataset[indices], (history_size, 1)))
    labels.append(np.reshape(dataset[i:i+target_size], (target_size, 1)))
  return np.array(data), np.array(labels)

def create_time_steps(length):
  return list(range(-length, 0))

def show_plot(plot_data, delta, title):
  labels = ['History', 'Model Prediction', 'True Future']
  marker = ['.-', '.-', '.-']
  time_steps = create_time_steps(plot_data[0].shape[0])
  if delta:
    future = delta
  else:
    future = 0

  plt.title(title)
  for i, x in enumerate(plot_data):
    if i:
      plt.plot(plot_data[i], marker[i], markersize=5,
               label=labels[i])
    else:
      plt.plot(time_steps, plot_data[i].flatten(), marker[i], label=labels[i])
  plt.legend()
  plt.xlim([time_steps[0], (future+8)*2])
  plt.xlabel('Time-Step')
  return plt

def df_by_store_prophet(df, store_no):
  df_store = df[df['Store'] == store_no]
  df_store = df_store.rename(columns = {'Date': 'ds', 'Weekly_Sales':'y'})
  df_example = df_store.groupby(by = 'ds').agg({'y':'sum'})
  df_example.index = pd.to_datetime(df_example.index) 
  df_example = df_example.asfreq(freq='7D')
  df_example = df_example.sort_index(ascending = True)
  df_example = df_example.fillna(value=0)
  return df_example
